In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps

import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
survey = Mag.readMagneticsObservations('./DIGHEM_Mag_floor10nt_25m.obs')
topo = np.genfromtxt('TKCtopoDwnS.dat', skip_header=1)
locs = survey.srcField.rxList[0].locs

# Build the problem
survey = ProblemSetter.setSyntheticProblem(locs, topo=topo, discretize=True, plotSections=True)


# 2.1.2.	Magnetic Data Gridding and Visualization#



In [2]:
ndat = survey.srcField.rxList[0].locs.shape[0]
# nC = int(nCx*nCy)

A = np.zeros((ndat, ndat))
for i in range(ndat):

    r = (locs[i, 0] - locs[:, 0])**2. + (locs[i, 1] - locs[:, 1])**2. +1e-8
    A[i, :] = r.T * (np.log((r.T)**0.5) - 1.)

In [3]:
np.mean(r**0.5)

2087.9196967580306

## DELETE THIS PROFILE SECTION (RETAINED IN NOTEBOOK 2.1.1)

In [4]:
param = Simulator.ViewMagSurveyWidget(survey)
display(param)

interactive(children=(FloatSlider(value=-2.2760687921291742e-10, continuous_update=False, description='East', max=2560.8134999998197, min=-2560.8135000002753, step=10.0), FloatSlider(value=-4.43486940293085e-11, continuous_update=False, description='North', max=2809.9999999999554, min=-2810.0000000000446, step=10.0), FloatSlider(value=90.0, continuous_update=False, description='Azimuth', max=180.0, step=5.0), FloatSlider(value=1900.9120036039603, continuous_update=False, description='Length', max=3801.8240072079207, min=20.0, step=20.0), BoundedFloatText(value=100.0, description='Sampling', max=1000.0, min=10.0, step=5.0), Output()), _dom_classes=('widget-interact',))

## Gridding the magnetic data

Magnetic data should be gridded prior to applying transforms and filters. This means that the flight line data must be interpolated on a regular 2D grid. There are numerous possible ways to grid data. Several researchers have demonstrated that minimum curvature methods are robust and yield a sensible and smooth result, so we have chosen minimum curvature to complete the gridding for this example. In this example, the data are interpolated on a 10 m x 10 m grid. The magnetic field parameters of central BC (Geoscience BC Search II area) are returned to here. 

The resulting gridded data shows the larger and smaller magnetic bodies, and the magnetic low related to the NW trending structure cross-cutting the larger magnetic domain.


In [5]:
view = Simulator.dataGriddingWidget(survey, fileName="DataGrid")
display(view)

interactive(children=(FloatText(value=10.0, description='Grid (m):'), Dropdown(description='Method', index=3, options=('nearest', 'linear', 'cubic', 'minimumCurvature'), value='minimumCurvature'), IntSlider(value=50, continuous_update=False, description='Contours', min=10, step=10), Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), ToggleButton(value=False, description='Export Grid', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))

In [6]:
# workDir = ''
# fName = 'DataGrid.tiff'

# dataGrid = DataIO.loadGeoTiffFile(workDir+fName, plotIt=True)

# Simulator.plotData2D(X,Y,dataGrid.values)

## Data visualization and image enhancement - MOVE VISUALIZATION TOOL HERE

Simple enhancements to the image can help an interpreter better visualize features within the magnetic data. Applying sun-shading from various angles, experimenting with colour contrast, and adding contour lines are a simple means to explore the data initially. 

Try using the slider bars above the map to change the direction (SunAzimuth) and angle (SunAngle) of sun shading, to vary color transparency (ColorTransp) and sun-shade transparency (HSTransp), change the vertical scale (vScale) and to manipulate magnetic contour lines (MagContour).

The color map or color 'look-up table' can also be changed. Some look-up tables will do an obviously better job of enhancing a range of features (those with a larger color range for example), while others may be used simply because of an interpreters personal preference. Several color look-up tables are available in the drop down menu.

Choice of color map is an often debated topic in geophysics! One important point to consider is that some color maps have the 'brightest' colors at the top of the scale (e.g. grey scale, viridis), whereas others have the brightest color in the mid-ranges (e.g. rainbow, jet, red-blue). This color ordering can be deceptive to the eye of an interpreter who is sensitive to color contrast. A color map with the brightest color in the mid-ranges and may cause problems if a map is printed in greyscale, suddenly mid-range values are the most brightest and clear features. Having bright colors in the mid-range however, may be useful for visually separating areas of high versus low magnetic response.       

Finally, the color stretch can be changed. Here we've allowed two options, a linear stretch (Linear), and a histogram equalized (HistEqualized) stretch. Using a linear stretch, the values within a range of magnetic data are mapped to 256 classes or bins within a chosen color map in a linear manner, meaning if there are few anomalously high values (outliers) in the range, the 'strongest' color values will be mapped rarely. If there are many middle-range values, the image may be flooded with mid range color values. While a linear color stretch will highlight the most anomalous values, it can wash out subtler features in the data. Histogram equialization will vary the color bin sizes to map an equal number of data points per bin, essentially giving equal weight to low, mid-range, and high magnetic values.        
references...
https://stats.stackexchange.com/questions/223315/why-use-colormap-viridis-over-jet